In [ ]:
#colab setup
!pip install timm
import timm

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastai.vision.all import *

In [ ]:
!pip install wandb -Uq
import wandb
wandb.login()

In [ ]:
sweep_config = {
    'method': 'grid',
    'metric': {'name': 'accuracy','goal': 'maximize'},
    'parameters': {
        'bs': {'values': [16, 64, 256, 1024]},
        'lr': {'values': [0.03, 0.001, 0.0001]},
        'aug_trfm': {'values': [False, True]},
        'epochs': {'value': 8},
    },
}
sweep_id = wandb.sweep(sweep_config, project="mnist_minimal_2")

In [ ]:
import gc
from fastai.callback.wandb import WandbCallback

def train(config=None):
    with wandb.init(config=config, group="minimal"):
        cfg = wandb.config
        dls = build_dls(cfg.bs, cfg.aug_trfm)
        learn_shallow = Learner(dls, ShallowNet(), metrics=accuracy, cbs=[MixedPrecision(), WandbCallback(log_preds=False)])
        learn_shallow.fit(cfg.epochs, lr=cfg.lr)

        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
wandb.agent(sweep_id, train)